In [ ]:
!pip install -e ./


In [ ]:
from ibge_gavb.localidades import *

In [ ]:
data = Estados()

In [ ]:
import requests
import urllib3
import ssl
import json


class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

In [ ]:
url = 'https://servicodados.ibge.gov.br/api/v1/localidades/regioes'
request = get_legacy_session().get(url)
json_ibge = json.loads(request.content.decode('utf-8'))

In [ ]:
import requests
import urllib3
import json

URL = 'https://servicodados.ibge.gov.br/api/v1/localidades/regioes'

class TestAdapter(requests.adapters.HTTPAdapter):
    def __init__(self, response):
        self._response = response
        super(TestAdapter, self).__init__()

    def send(self, request, *args, **kwargs):
        return self.build_response(request, self._response)

resp = urllib3\
    .HTTPResponse(body=json.dumps({'results': results}), status=200,
                  headers={'content-type': 'application/json'})
adapter = TestAdapter(resp)
session = requests.Session()
session.mount(URL, adapter)

response = session.post(URL)
response.json()

In [1]:
"""
API do IBGE
https://servicodados.ibge.gov.br/api/docs/

"""

import requests
import json
import urllib3
import ssl


headers = {
    "Content-Type": "application/json;charset=UTF-8",
    "User-Agent": "ibge_gavb.py - https://github.com/GAVB-SERVICOS/ibge_gavb",
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
}


class CustomHttpAdapter(requests.adapters.HTTPAdapter):
    """
    Essa classe é responsável por transportar o adapter que permite usar
    o ssl_context padrão evitando o erro unsafe legacy renegotiation disabled.
    """

    def __init__(self, **kwargs):
        """
        Inicia o ssl_context.

        :param Kwargs: Parâmetro extra
        :type: any

        """
        ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        ctx.options |= 2186412244  # 0x4 OP_LEGACY_SERVER_CONNECT
        self.ssl_context = ctx
        super().__init__(**kwargs)

    def init_poolmanager(self, connections: int, maxsize: int, block: bool = False):
        """
        Configura a PoolManager.

        :param conections: Número de conexões pool para armazenar
                           em cache antes de descartar o mínimo.
        :type: int
        :param maxsize: Método da classe PoolManager.
        :type: int
        :param block: Método da classe PoolManager.
        :type: bool

        """

        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections,
            maxsize=maxsize,
            block=block,
            ssl_context=self.ssl_context,
        )


def get_legacy_session():
    """
    Função responsável por abrir a sessão com o certificado
    ssl criando uma autenticação.
    """

    session = requests.session()
    session.mount("https://", CustomHttpAdapter())
    return session

In [33]:
class Municipio(object):
    """
    Classe que conecta a API IBGE acessando o arquivo
    json trazendo o conteúdo do Município específico.
    """

    def __init__(self, codigo_ibge:str=None):
        """
        Faz o request e gera conteúdo em json.
        :param codigo_ibge: Número do código do município.
        :type: str
        """
        url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios/{}"
        # request = requests.get(url.format(codigo_ibge), headers=headers)
        request = get_legacy_session().get(url.format(codigo_ibge), headers=headers)
        self.json_ibge = json.loads(request.content.decode("utf-8"))

    def json(self):
        """
        Retorna o conteúdo.

        :return: String com o conteúdo
        :rtype: str
        """
        return self.json_ibge

    def __repr__(self):
        """
        Representação da api em json.

        :return: String com o conteúdo
        :rtype: str
        """
        return repr(str(self.json()))

    def count(self):
        """
        Retorna o tamanho do conteúdo.

        :return: Inteiro reprentando o tamanho do json. 
        :rtype: int
        """
        return int(len(self.json_ibge) / 3)

    def getId(self):
        """
        Retorna os IDs.

        :return: Lista com os ids 
        :rtype: list
        """
        return self.json_ibge["id"]

    def getNome(self):
        return self.json_ibge["nome"]

    def getDescricaoUF(self):
        return self.json_ibge["microrregiao"]["mesorregiao"]["UF"]["nome"]

    def getUF(self):
        return self.json_ibge["microrregiao"]["mesorregiao"]["UF"]["sigla"]

In [34]:
data = Municipio('5221858')

In [39]:
data.getDescricaoUF()


'Goiás'